# SUDOKU SOLVER

Importing the relevant libraries:

In [1]:
# Including path to previous directory in built-in variable sys.path

import sys

sys.path.append('../')

In [3]:
import numpy as np
from sudoku.utils import N, STEP, objective_grid

Definition of auxiliary functions:

In [4]:
def square_loc(pos):  # pos corresponds to x or y
    pos_square = pos // STEP * STEP
    
    return pos_square, pos_square + STEP

## Defining CSP

Class that defines the CSP and allows the application of the solver:

In [5]:
class SudokuCSP:
    def __init__(self, grid):
        self.domains = self.node_consistency(grid)
                
        self.neighbors = self.generate_neighbors()  # avoid checking each time it is necessary
        
        # Similar difference constraints, so attribute contains only the scopes
        self.constraints = [(var, neighbor) for var in self.vars for neighbor in self.neighbors[var]]
        
        self.cuts = {var : [] for var in self.vars}
                
    @staticmethod
    def node_consistency(grid):
        domains = {(y, x) : [grid[y, x]] if grid[y, x] else list(range(1, N+1)) 
                   for y in range(N)
                   for x in range(N)}
        
        return domains
    
    def generate_neighbors(self):
        neighbors = {(y, x) : set([(y, i) for i in range(N) if i != x] + 
                                  [(j, x) for j in range(N) if j != y] + 
                                  [(j, i) for j in range(*square_loc(y)) for i in range(*square_loc(x))
                                   if i != x and j != y])
                     for y, x in self.vars}
        
        return neighbors
    
    def ac_3(self):
        queue = self.constraints.copy()
        
        while queue:
            xi, xj = queue.pop(0)
            
            if self.revise(xi, xj):
                if not self.domains[xi]:
                    return False
                
                for xk in self.neighbors[xi] - {xj}:
                    queue.append((xk, xi))
        
        return True
    
    def revise(self, xi, xj):
        revised = False
        
        for x in self.domains[xi]:
            if not any([x != y for y in self.domains[xj]]):
                self.domains[xi].remove(x)
                
                revised = True
        
        return revised
        
    @property
    def vars(self):
        return list(self.domains.keys())
    
    @property
    def won(self):
        if all(len(domain) == 1 for domain in csp.domains.values()):
            return objective_grid(np.array([domain[0] for domain in csp.domains.values()]).reshape((N, N)))
        
        return False

Implementing the heuristics used in backtracking search for csp and auxiliary functions:

In [6]:
# Implementing the functions that apply the heuristics

def select_unassigned_variable(assignment, csp):
    available_vars = [var
                      for var in csp.vars
                      if var not in assignment.keys()]
    
    # mrv heuristic in the key parameter chooses the variable with the smallest legal value
    return min(available_vars, key=lambda var: len(csp.domains[var]))

def order_domain_values(assignment, csp, var):
    # Implementing the least constraining value heuristics
    lcv = lambda value: len([neighbor
                             for neighbor in csp.neighbors[var]
                             if neighbor not in assignment and value in csp.domains[neighbor]])
    
    return sorted(csp.domains[var], key=lcv)

# Auxiliary functions

def is_consistent(assignment, csp, var, value):
    for current_var, current_value in assignment.items():
        if current_value == value and current_var in csp.neighbors[var]:
            return False
        
    return True

def assign(assignment, csp, var, value):
    assignment[var] = value
    
    # Making inferences
    for neighbor in csp.neighbors[var]:
        if value in csp.domains[neighbor] and neighbor not in assignment.keys():
            csp.domains[neighbor].remove(value)
            
            csp.cuts[var].append((neighbor, value))

def unassign(assignment, csp, var):
    if var in assignment.keys():
        # Unmaking inferences
        for neighbor, value in csp.cuts[var]:
            csp.domains[neighbor].append(value)
        
        csp.cuts[var].clear()
        
        del assignment[var]
        
def assignment_in_grid(assignment):
    grid = np.zeros((N, N))

    for pos, value in assignment.items():
        grid[pos] = value

    return grid

Implementing backtracking search:

In [7]:
def backtracking_search(csp):
    return backtrack({}, csp)

def backtrack(assignment, csp):
    if len(assignment) == len(csp.vars):
        return assignment

    var = select_unassigned_variable(assignment, csp)

    for value in order_domain_values(assignment, csp, var):
        if is_consistent(assignment, csp, var, value):
            assign(assignment, csp, var, value)

            result = backtrack(assignment, csp)
            if result:
                return result

        unassign(assignment, csp, var)

    return None

Testing class instantiation and resolution through AC-3:

In [8]:
# Sudoku that can only be solved with AC-3
grid_ac3 = np.array([[1, 5, 7, 4, 0, 8, 9, 3, 0],
                     [8, 0, 3, 0, 0, 5, 0, 0, 4],
                     [6, 0, 4, 0, 1, 9, 8, 0, 7],
                     [0, 6, 0, 7, 0, 0, 0, 0, 0],
                     [0, 0, 0, 6, 0, 4, 2, 0, 3],
                     [0, 3, 9, 5, 0, 1, 0, 0, 0],
                     [5, 0, 1, 8, 0, 0, 3, 0, 9],
                     [3, 8, 0, 0, 5, 6, 0, 4, 0],
                     [0, 0, 6, 0, 3, 7, 0, 2, 0]], dtype='int8')

In [9]:
csp = SudokuCSP(grid_ac3)

csp.domains

{(0, 0): [1],
 (0, 1): [5],
 (0, 2): [7],
 (0, 3): [4],
 (0, 4): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (0, 5): [8],
 (0, 6): [9],
 (0, 7): [3],
 (0, 8): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 0): [8],
 (1, 1): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 2): [3],
 (1, 3): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 4): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 5): [5],
 (1, 6): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 7): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 8): [4],
 (2, 0): [6],
 (2, 1): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (2, 2): [4],
 (2, 3): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (2, 4): [1],
 (2, 5): [9],
 (2, 6): [8],
 (2, 7): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (2, 8): [7],
 (3, 0): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (3, 1): [6],
 (3, 2): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (3, 3): [7],
 (3, 4): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (3, 5): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (3, 6): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (3, 7): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (3, 8): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (4, 0): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (4, 1): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (4, 2): [1,

In [10]:
# Performing the propagation of inferences:

csp.ac_3()

True

In [11]:
csp.won

True

In [12]:
# AC-3 solved the problem, so the search returns a valid assignment quickly

assignment = backtracking_search(csp)

grid = assignment_in_grid(assignment)
    
print(grid)
print(objective_grid(grid))

[[1. 5. 7. 4. 6. 8. 9. 3. 2.]
 [8. 9. 3. 2. 7. 5. 6. 1. 4.]
 [6. 2. 4. 3. 1. 9. 8. 5. 7.]
 [4. 6. 8. 7. 2. 3. 1. 9. 5.]
 [7. 1. 5. 6. 9. 4. 2. 8. 3.]
 [2. 3. 9. 5. 8. 1. 4. 7. 6.]
 [5. 7. 1. 8. 4. 2. 3. 6. 9.]
 [3. 8. 2. 9. 5. 6. 7. 4. 1.]
 [9. 4. 6. 1. 3. 7. 5. 2. 8.]]
True


Testing class instantiation and resolution via backtracking search:

In [13]:
# Sudoku that cannot be solved with AC-3 alone
grid_backtracking = np.array([[0, 0, 5, 3, 0, 0, 0, 0, 0],
                              [8, 0, 0, 0, 0, 0, 0, 2, 0],
                              [0, 7, 0, 0, 1, 0, 5, 0, 0],
                              [4, 0, 0, 0, 0, 5, 3, 0, 0],
                              [0, 1, 0, 0, 7, 0, 0, 0, 6],
                              [0, 0, 3, 2, 0, 0, 0, 8, 0],
                              [0, 6, 0, 5, 0, 0, 0, 0, 9],
                              [0, 0, 4, 0, 0, 0, 0, 3, 0],
                              [0, 0, 0, 0, 0, 9, 7, 0, 0]], dtype='int8')

In [14]:
csp = SudokuCSP(grid_backtracking)

csp.domains

{(0, 0): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (0, 1): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (0, 2): [5],
 (0, 3): [3],
 (0, 4): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (0, 5): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (0, 6): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (0, 7): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (0, 8): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 0): [8],
 (1, 1): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 2): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 3): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 4): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 5): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 6): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (1, 7): [2],
 (1, 8): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (2, 0): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (2, 1): [7],
 (2, 2): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (2, 3): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (2, 4): [1],
 (2, 5): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (2, 6): [5],
 (2, 7): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (2, 8): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (3, 0): [4],
 (3, 1): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (3, 2): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (3, 3): [1, 2, 3, 4, 5, 6, 7, 8, 9],
 (3, 4): [1, 2

In [15]:
csp.ac_3()

True

In [16]:
csp.won

False

In [17]:
assignment = backtracking_search(csp)

grid = assignment_in_grid(assignment)
    
print(grid)
print(objective_grid(grid))

[[1. 4. 5. 3. 2. 7. 6. 9. 8.]
 [8. 3. 9. 6. 5. 4. 1. 2. 7.]
 [6. 7. 2. 9. 1. 8. 5. 4. 3.]
 [4. 9. 6. 1. 8. 5. 3. 7. 2.]
 [2. 1. 8. 4. 7. 3. 9. 5. 6.]
 [7. 5. 3. 2. 9. 6. 4. 8. 1.]
 [3. 6. 7. 5. 4. 2. 8. 1. 9.]
 [9. 8. 4. 7. 6. 1. 2. 3. 5.]
 [5. 2. 1. 8. 3. 9. 7. 6. 4.]]
True
